In [1]:
import pandas as pd
import numpy as np

from rutas import *

%matplotlib inline

#kaggle competitions submissions -c navent
#kaggle competitions submit -c navent -f submit-.csv -m "Submit message"

In [2]:
DATASET_SAMPLE_SIZE = 500000

#### Generar 500.000 postulaciones y 500.000 no postulaciones (que provienen de avisos vistos pero no postulados)

In [3]:
postulaciones = pd.read_csv(RUTA_POSTULACIONES)

In [4]:
postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones['fechapostulacion'], format='%Y-%m-%d %H:%M:%S')

In [5]:
vistas = pd.read_csv(RUTA_VISTAS).rename(columns={'idAviso': 'idaviso'})

In [6]:
vistas['timestamp'] = pd.to_datetime(vistas['timestamp']) #2018-02-23T13:38:13.187-0500

In [7]:
min(vistas['timestamp'])

Timestamp('2018-02-23 18:38:10.808000')

In [8]:
max(vistas['timestamp'])

Timestamp('2018-05-01 03:59:59.976000')

In [9]:
min(postulaciones['fechapostulacion'])

Timestamp('2018-01-15 00:00:01')

In [10]:
max(postulaciones['fechapostulacion'])

Timestamp('2018-04-17 10:42:39')

In [11]:
min_fecha = max(pd.Timestamp('2018-02-23 18:38:10.808000'), pd.Timestamp('2018-01-15 00:00:01'))
max_fecha = min(pd.Timestamp('2018-04-17 10:42:39'), pd.Timestamp('2018-05-01 03:59:59.976000'))
mascara_temporal = lambda df, col: (df[col] > min_fecha) & (df[col] < max_fecha)

In [14]:
postulaciones_en_periodo = postulaciones[mascara_temporal(postulaciones, 'fechapostulacion')].drop(axis=1, labels=['fechapostulacion']).drop_duplicates()
vistas_en_periodo = vistas[mascara_temporal(vistas, 'timestamp')].drop(axis=1, labels=['timestamp']).drop_duplicates()

In [15]:
len(postulaciones_en_periodo)

3519581

In [16]:
# Eliminar las vistas que se convirtieron en postulaciones
vistas_no_postuladas = pd.merge(vistas_en_periodo, postulaciones_en_periodo[['idaviso', 'idpostulante']], on=['idaviso', 'idpostulante'], how='left', indicator=True)
vistas_no_postuladas = vistas_no_postuladas[~(vistas_no_postuladas['_merge'] == 'both')]
vistas_no_postuladas = vistas_no_postuladas.drop(axis=1, labels=['_merge'])

In [17]:
len(vistas_no_postuladas)

2447237

In [18]:
postulaciones_dataset = postulaciones_en_periodo.sample(n=DATASET_SAMPLE_SIZE)

In [19]:
vistas_dataset = vistas_no_postuladas.sample(n=DATASET_SAMPLE_SIZE)

In [20]:
postulaciones_dataset['sepostulo'] = 1
vistas_dataset['sepostulo'] = 0

In [21]:
postulaciones_dataset = postulaciones_dataset[['idaviso', 'idpostulante', 'sepostulo']]
vistas_dataset = vistas_dataset[['idaviso', 'idpostulante', 'sepostulo']]

In [22]:
df = pd.merge(postulaciones_dataset, vistas_dataset, on=['idpostulante', 'idaviso'], how='outer')

In [23]:
len(df)

1000000

In [24]:
df['sepostulo_x'] = df['sepostulo_x'].fillna(0).astype('int')
df = df.drop(axis=1, labels=['sepostulo_y']).rename(columns={'sepostulo_x': 'sepostulo'})

In [25]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,idaviso,idpostulante,sepostulo
0,1112370921,LNKj9Jr,1
1,1112353843,YdDO2V,0
2,1112448587,6QY4DR,0
3,1112468192,wVYmGBL,0
4,1112451122,aa1LaN,0


In [26]:
df.to_csv('/home/luciano/orga-datos/training-set-no-random.csv', index=False)

#### Generar 1.000.000 de pares aleatorios de no-postulaciones, verificando que no estén en fiuba_4_postulaciones.csv

Estos pares luego van ser pasados por un random forest para revisar cuales realmente no van a ser postulados y reducidos de 1.000.000 a 500.000

In [6]:
avisos = pd.read_csv(RUTA_AVISOS_DETALLE, usecols=['idaviso'])

In [7]:
postulantes = pd.read_csv(RUTA_POSTULANTES, usecols=['idpostulante'])

In [10]:
(len(avisos) * len(postulantes)) - len(postulaciones)

12748839682

In [11]:
# En total hay 12.700 millones de pares posibles

In [86]:
RANDOM_SEED = 12
np.random.seed(RANDOM_SEED)
rnd_keys = np.random.rand(1100000, 2)

In [87]:
rnd_keys[:, 0] = rnd_keys[:, 0] * len(avisos)

In [88]:
rnd_keys[:, 1] = rnd_keys[:, 1] * len(postulantes)

In [89]:
rnd_keys = pd.DataFrame(rnd_keys, columns=['idaviso', 'idpostulante'])
rnd_keys['idaviso'] = rnd_keys['idaviso'].astype('int')
rnd_keys['idpostulante'] = rnd_keys['idpostulante'].astype('int')
assert len(rnd_keys.drop_duplicates()) > 1000000

In [90]:
no_postulados = pd.DataFrame()

In [91]:
no_postulados['idaviso'] = pd.merge(rnd_keys[['idaviso']], avisos, left_on='idaviso', right_index=True, how='inner')['idaviso_y']

In [92]:
no_postulados = no_postulados.sample(frac=1).reset_index(drop=True)

In [93]:
idpostulantes = pd.merge(rnd_keys[['idpostulante']], postulantes, left_on='idpostulante', right_index=True, how='inner')

In [94]:
idpostulantes = idpostulantes[['idpostulante_y']].sample(frac=1).reset_index(drop=True).rename(columns={'idpostulante_y': 'idpostulante'})

In [95]:
no_postulados = pd.merge(no_postulados, idpostulantes, left_index=True, right_index=True, how='inner')

In [98]:
no_postulados = pd.merge(no_postulados, postulaciones[['idaviso', 'idpostulante']], on=['idaviso', 'idpostulante'], how='left', indicator=True)

In [99]:
no_postulados = no_postulados[no_postulados['_merge'] == 'left_only']

In [101]:
no_postulados = no_postulados[:1000000].drop(axis=1, labels=['_merge'])

In [104]:
no_postulados.to_csv('/home/luciano/orga-datos/training-set-no-postulados-random.csv', index=False)